email agent
- authenticates user
    - only then are they allowed into the "inbox"
    - dynamic tools and prompt on the condition of there being an email and password in state that match hardcoded
- checks "inbox"
    - email in tool
- sends emails
    - human in the loop

In [1]:
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
from dataclasses import dataclass

@dataclass
class EmailContext:
    email_address: str = "julie@example.com"
    password: str = "password123"

In [3]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated: bool

In [4]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(update={
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

In [25]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools) 
    return handler(request)

In [26]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [38]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

agent = create_agent(
    "gpt-5-nano",
    tools=[authenticate, check_inbox, send_email],
    checkpointer=InMemorySaver(),
    state_schema=AuthenticatedState,
    context_schema=EmailContext,
    middleware=[
        dynamic_tool_call, 
        dynamic_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True,
            })
        ]
    )


In [41]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "11"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="Draft A")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

In [42]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi Jane, that sounds great! I’d love to catch up. How about Tuesday at 3 PM or Thursday morning? Let me know what works for you.

- Julie


In [43]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

print(response["messages"][-1].content)

Done. I sent Draft A to jane@example.com with the subject “Re: Coffee next week.” Message sent:

Hi Jane, that sounds great! I’d love to catch up. How about Tuesday at 3 PM or Thursday morning? Let me know what works for you.

- Julie

Would you like me to:
- Mark Jane’s original message as read, archive, or delete?
- Check for more emails in your inbox?
- Draft a follow-up reply or adjust the time options (e.g., add specific times)?
- Create a calendar reminder or invite once Jane replies?


In [44]:
from pprint import pprint

pprint(response)

{'authenticated': True,
 'messages': [HumanMessage(content='Hi there please check my inbox', additional_kwargs={}, response_metadata={}, id='e4cbd668-0184-4c11-b321-cc85c18d3454'),
              AIMessage(content='I can help with that, but I need to sign you in first.\n\nPlease provide:\n- Email\n- Password\n\nNote: I’ll only use these credentials to access your inbox for this session and won’t share them. If you’d prefer not to share credentials here, I can guide you through steps to securely reset your password or access your inbox another way.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 658, 'prompt_tokens': 150, 'total_tokens': 808, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, '